# 趣味赛车 - 平行停车
在本节中，你将要编写一个函数，用于实现平行停放车辆所需的正确步骤。

运行之后，你会打开一个新的窗口来运行你的小车。

你可以使用键盘上的上下左右键来控制你的小车，体验小车运动的感觉。

你需要提交的的项目，是使用:
- car.accelerate()
- car.deaccelerate()
- car.steerright()
- car.steerleft()
来控制小车，让他成功倒车进入两车中间；
你的小车如果撞上停止的车辆，小车会因为撞击而爆炸；

**如需退出窗口，请使用键盘ESC**

**SPACE可以帮助你重启这个项目**

![](https://upload.wikimedia.org/wikipedia/commons/2/26/ParallelParkingAnimation.gif)

In [1]:
#initialize the screen
import pygame, math, sys
from pygame.locals import *

def level3():
    pygame.init()
    screen = pygame.display.set_mode((800,600))
    #GAME CLOCK
    clock = pygame.time.Clock()
    font = pygame.font.Font(None, 30)
    running = True
    win_text = font.render('', True, (0, 255, 0))
    
    def rot_center(image, rect, angle):
        """rotate an image while keeping its center"""
        rot_image = pygame.transform.rotate(image, angle)
        rot_rect = rot_image.get_rect(center=rect.center)
        return rot_image,rot_rect
    
    class Background(pygame.sprite.Sprite):
        def __init__(self, image_file, location):
            pygame.sprite.Sprite.__init__(self)  #call Sprite initializer
            self.image = pygame.image.load(image_file)
            self.rect = self.image.get_rect()
            self.rect.left, self.rect.top = location
    
    class CarSprite(pygame.sprite.Sprite):
        def __init__(self, image, x, y):            
            pygame.sprite.Sprite.__init__(self)
            self.image = pygame.image.load(image)
            self.rect = self.image.get_rect()
            self.image_orig = self.image
            self.screen = pygame.display.get_surface()
            self.area = self.screen.get_rect()
            self.x = x
            self.y = y
            self.rect.topleft = self.x, self.y
            #self.x, self.y = findspawn()
            self.dir = 0
            self.speed = 0.0
            self.maxspeed = 11.5
            self.minspeed = -1.85
            self.acceleration = 0.095
            self.deacceleration = 0.12
            self.softening = 0.04
            self.steering = 1.60
        
        def update(self, deltat):
            self.x = self.x + self.speed * math.cos(math.radians(270-self.dir))
            self.y = self.y + self.speed * math.sin(math.radians(270-self.dir))
            self.rect = self.image.get_rect()
            self.rect.center = (self.x, self.y)
        
        def impact(self):
            if self.speed > 0:
                self.speed = self.minspeed

        def soften(self):
            if self.speed > 0:
                self.speed -= self.softening
            if self.speed < 0:
                self.speed += self.softening

    #Accelerate the vehicle
        def accelerate(self):
            if self.speed < self.maxspeed:
                self.speed = self.speed + self.acceleration

    #Deaccelerate.
        def deaccelerate(self):
            if self.speed > self.minspeed:
                self.speed = self.speed - self.deacceleration

    #Steer.
        def steerleft(self):
            self.dir = self.dir+self.steering
            if self.dir > 360:
                self.dir = 0
            self.image, self.rect = rot_center(self.image_orig, self.rect, self.dir)

    #Steer.
        def steerright(self):
            self.dir = self.dir-self.steering
            if self.dir < 0:
                self.dir = 360  
            self.image, self.rect = rot_center(self.image_orig, self.rect, self.dir)
        
    class PadSprite(pygame.sprite.Sprite):
        normal = pygame.image.load('blank.png')
        def __init__(self, x, y):
            super(PadSprite, self).__init__()
            self.rect = pygame.Rect(self.normal.get_rect())
            self.rect.center = (x, y)
            self.image = self.normal
            self.rect.x = x
            self.rect.y = y
        def update(self):
            self.image = self.normal
    pads = [
        PadSprite(500, 20),
        PadSprite(500, 450)
    ]
    pad_group = pygame.sprite.RenderPlain(*pads)

    # CREATE A CAR AND RUN
    rect = screen.get_rect()
    car = CarSprite('car_1.png', 400, 100)
    car_group = pygame.sprite.RenderPlain(car)
    BackGround = Background('NEAREST.jpg', [0,0])

    #THE GAME LOOP
    while running:
        #USER INPUT

        deltat = clock.tick(60)
        for event in pygame.event.get():
            if not hasattr(event, 'key'): continue
            if event.key == K_ESCAPE: 
                pygame.quit()
                running = False
                exit()
 
        if running:
            # This is about keyboard!
            keys = pygame.key.get_pressed()                
            if keys[K_LEFT]:
                car.steerleft()
            if keys[K_RIGHT]:
                car.steerright()
            if keys[K_UP]:
                car.accelerate()
            else:
                car.soften()
            if keys[K_DOWN]:
                car.deaccelerate()
                
            x = car.x
            y = car.y
            ## your code here!!
            
            
            
            
            #RENDERING
            #screen.fill((0,0,0))
            screen.fill([255, 255, 255])
            screen.blit(BackGround.image, BackGround.rect)
            text_x = 'x: {:.3f}'.format(car.x)
            text_y = 'y: {:.3f}'.format(car.y)
            win_text_x = font.render(text_x, True, (0, 255, 0))
            win_text_y = font.render(text_y, True, (0, 255, 0))
            for pad in pads:
                if car.rect.colliderect(pad.rect):
                    timer_text = font.render("Crash! Press SPACE TO RESTART!", True, (255,0,0))
                    screen.blit(timer_text, (300, 300))
                    car.kill()
                    
            if keys[K_SPACE]:
                car = CarSprite('car_1.png', 400, 100)
                car_group = pygame.sprite.RenderPlain(car)
                timer_text = font.render("", True, (255,0,0))
                screen.blit(timer_text, (300, 300))
            
            car_group.update(deltat)
            car_group.draw(screen)
            #screen.blit(car.image, car.rect)
            pad_group.draw(screen)
            #pygame.draw.rect(screen,(255, 200, 0), car.rect)
            screen.blit(win_text_x, (50, 50))
            screen.blit(win_text_y, (50, 70))

            #Counter Render
            pygame.display.flip()


pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
level3()